In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,                                                
                                                      random_state=0)

# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

num_cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
print(num_cols_with_missing)

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Approach 1: Drop columns with missing values
reduced_X_train = X_train.drop(num_cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(num_cols_with_missing, axis=1)
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

# Approach 2: Imputation with mean (default)
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))
imputed_X_test = pd.DataFrame(imputer.transform(X_test))

# Restore column names after imputation
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
imputed_X_test.columns = X_test.columns

print("MAE (Imputation with mean):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

# Train final model with mean imputation
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(imputed_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(imputed_X_valid)
print("MAE (Mean imputation approach):")
print(mean_absolute_error(y_valid, preds_valid))

# Approach 3: Imputation with median
median_imputer = SimpleImputer(strategy='median')
median_X_train = pd.DataFrame(median_imputer.fit_transform(X_train))
median_X_valid = pd.DataFrame(median_imputer.transform(X_valid))
median_X_test = pd.DataFrame(median_imputer.transform(X_test))

# Restore column names after imputation
median_X_train.columns = X_train.columns
median_X_valid.columns = X_valid.columns
median_X_test.columns = X_test.columns

print("MAE (Imputation with median):")
print(score_dataset(median_X_train, median_X_valid, y_train, y_valid))

# Train model with median imputation for final predictions
final_model = RandomForestRegressor(n_estimators=100, random_state=0)
final_model.fit(median_X_train, y_train)

# Get test predictions
preds_test = final_model.predict(median_X_test)

# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission2.csv', index=False)

# ===== CATEGORICAL DATA HANDLING =====
# Reset to use full dataset including categorical variables
X = X_full  # Include categorical variables
X_test = X_test_full

# Break off validation set again (now with categorical data)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

# Approach 1: Drop categorical variables
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
drop_X_train = X_train.drop(object_cols, axis=1)
drop_X_valid = X_valid.drop(object_cols, axis=1)

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

# Check unique values in categorical columns
print("Unique values in 'Condition2' column in training data:", X_full['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_full.loc[X_valid.index, 'Condition2'].unique())

# Approach 2: Ordinal Encoding
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols) - set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply ordinal encoder only if there are columns to encode
if good_label_cols:
    ordinal_encoder = OrdinalEncoder()
    label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
    label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

print("MAE from Approach 2 (Ordinal Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
print("Unique values per categorical column:")
print(sorted(d.items(), key=lambda x: x[1]))

# Approach 3: One-Hot Encoding
# Columns that will be one-hot encoded (low cardinality)
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset (high cardinality)
high_cardinality_cols = list(set(object_cols) - set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64
['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
MAE (Drop columns with missing values):
17837.82570776256
MAE (Imputation with mean):
18062.894611872147
MAE (Mean imputation approach):
18062.894611872147
MAE (Imputation with median):
17791.59899543379
MAE from Approach 1 (Drop categorical variables):
18206.697694063925
Unique values in 'Condition2' column in training data: ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']
Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heatin

In [43]:
# Train final model
final_model = RandomForestRegressor(n_estimators=100, random_state=0)
final_model.fit(final_X_train, y_train)  # Use whatever your final processed data is called

# Make predictions on test data (make sure X_test is processed the same way)
test_predictions = final_model.predict(final_X_test)

# Create submission file
submission3 = pd.DataFrame({
    'Id': X_test.index,
    'SalePrice': test_predictions
})

submission3.to_csv('submission.csv', index=False)
print("Submission file created!")

Submission file created!


c:\Users\mussa\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [44]:
import os
print("Current directory:", os.getcwd())
print("Files in directory:", os.listdir('.'))

Current directory: c:\Users\mussa\OneDrive\Desktop\Projects\Kaggle Machine learning\ML -2
Files in directory: ['Exercise.ipynb', 'Exercise1.ipynb', 'melb_data.csv', 'melb_pred.ipynb', 'submission.csv', 'submission2.csv', 'test.csv', 'train.csv']


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')